<a href="https://colab.research.google.com/github/Joycexu602/Credit_Risk_Analysis/blob/main/657_inclass_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-Up and Load Data

In [4]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from sklearn import metrics
from keras.layers import Embedding
from keras.layers import concatenate
from keras.layers import Reshape

In [5]:
dat = pd.read_csv("HealthData.txt", sep=" ")
dat.head()

,year,age,female,race,married,edu,income,msa,region,limitation,chronic,smoke,industry,occupation,uninsured,erexp,ernum
0,2010,28,0,1,1,14,67000,1,1,0,3,0,6,7,0,0,0
1,2010,25,1,1,1,14,67000,1,1,0,1,0,-1,-1,0,0,0
2,2010,51,0,6,1,17,91420,1,4,0,0,0,10,2,0,0,0
3,2010,53,1,4,1,17,91420,1,4,0,0,0,10,2,0,0,0
4,2010,33,0,1,1,12,19500,1,3,0,0,1,11,7,1,0,0


# **Question 1**

**Split data training and test sets, using observations in year 2010 as training and observations in year 2011 as test. Show the number of observations in each set.**

In [48]:
train = dat[dat['year'] == 2010]
test = dat[dat['year'] == 2011]
print(train.shape)
print(test.shape)

(17847, 17)
(19134, 17)


**There are 17,847 observations in the training dataset, and there are 19,134 observations in the test dataset.**

# **Question 2**

**Fit a Poisson neural network using three inputs: age, income (in log scale), and industry. Start with tuning parameters: (1) embedding dimension = 1; (2) number of hidden layers = 1; (3) number of neurons = 2. Calculate test data MSE.**

In [49]:
# Fit a Poisson Neural Net
# convert to float
convert_dict = {'ernum': float,
                'chronic': float,
                'income': float
                }
train = train.astype(convert_dict)
test = test.astype(convert_dict)

In [50]:
# creat new variable
train['logincome'] = np.log(train['income'])
test['logincome'] = np.log(test['income'])

In [51]:
xnames = ['age','logincome','industry']
ynames = ['ernum']
xtrain = train[xnames]
ytrain = train[ynames]

In [52]:
in_layer = Input(shape = (xtrain.shape[1],))
dense1 = Dense(2, activation = 'tanh')(in_layer)
mu = Dense(1, activation = 'exponential')(dense1)

In [53]:
model = Model(inputs = in_layer, outputs = mu)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_6 (Dense)             (None, 2)                 8         
                                                                 
 dense_7 (Dense)             (None, 1)                 3         
                                                                 
Total params: 11 (44.00 Byte)
Trainable params: 11 (44.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [55]:
model.fit(xtrain, ytrain, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 1s - loss: 0.6498 - 748ms/epoch - 5ms/step
Epoch 2/100
140/140 - 0s - loss: 0.4655 - 197ms/epoch - 1ms/step
Epoch 3/100
140/140 - 0s - loss: 0.4373 - 201ms/epoch - 1ms/step
Epoch 4/100
140/140 - 0s - loss: 0.4358 - 208ms/epoch - 1ms/step
Epoch 5/100
140/140 - 0s - loss: 0.4345 - 190ms/epoch - 1ms/step
Epoch 6/100
140/140 - 0s - loss: 0.4333 - 221ms/epoch - 2ms/step
Epoch 7/100
140/140 - 0s - loss: 0.4324 - 224ms/epoch - 2ms/step
Epoch 8/100
140/140 - 0s - loss: 0.4316 - 190ms/epoch - 1ms/step
Epoch 9/100
140/140 - 0s - loss: 0.4311 - 195ms/epoch - 1ms/step
Epoch 10/100
140/140 - 0s - loss: 0.4307 - 202ms/epoch - 1ms/step
Epoch 11/100
140/140 - 0s - loss: 0.4304 - 209ms/epoch - 1ms/step
Epoch 12/100
140/140 - 0s - loss: 0.4302 - 198ms/epoch - 1ms/step
Epoch 13/100
140/140 - 0s - loss: 0.4300 - 203ms/epoch - 1ms/step
Epoch 14/100
140/140 - 0s - loss: 0.4299 - 197ms/epoch - 1ms/step
Epoch 15/100
140/140 - 0s - loss: 0.4297 - 198ms/epoch - 1ms/step
Epoch 16/100
140/14

In [56]:
xtest = test[xnames]
ytest = test[ynames]
yhat = model.predict(xtest)
print(yhat)

598/598 [==============================] - 1s 1ms/step
[[0.11939447]
 [0.14527753]
 [0.11939447]
 ...
 [0.11939447]
 [0.11939502]
 [0.11939447]]


In [60]:
metrics.mean_squared_error(ytest, yhat)

0.34191002214413274

**The test data MSE equals to 0.3419.**

# **Question 3**

**From 2, change embedding dimension to 2, does this improve test data MSE?**

In [61]:
# Categorical Embedding
# label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['industry'] = le.fit_transform(train.industry.values)
train['industry'].value_counts()

industry
0     5358
10    2967
5     1634
9     1401
4     1285
11    1167
3      793
8      706
13     650
12     642
6      631
7      262
1      192
14      67
15      53
2       39
Name: count, dtype: int64

In [62]:
test['industry'] = le.fit_transform(test.industry.values)
test['industry'].value_counts()

industry
0     5684
10    3188
5     1777
9     1553
11    1325
4     1317
3      856
8      729
12     720
13     715
6      643
7      255
1      207
14      61
15      54
2       50
Name: count, dtype: int64

In [63]:
# define data
numVar = ['age','logincome']
catVar = ['industry']
xnum_train = train[numVar]
xcat_train = train[catVar]
x_train = [xnum_train, xcat_train]
y_train = train['ernum']

from keras.layers import Embedding
from keras.layers import concatenate
from keras.layers import Reshape

# define the NN structure
# define numerical inputs
in_num_layer = Input(shape = (xnum_train.shape[1],))
in_cat_layer = Input(shape = (1,))

# define embedding layers
embed_size = 2 # embedding size
em_layer = Embedding(input_dim = len(np.unique(xcat_train)), output_dim = embed_size)(in_cat_layer)
em_layer = Reshape(target_shape=(embed_size,))(em_layer)

in_layers = [in_num_layer,in_cat_layer]

# concatenate emebedding layer and numeric input layer together
conc_all = concatenate([in_num_layer, em_layer])
dense1 = Dense(2, activation='relu')(conc_all)
mu = Dense(1, activation = 'exponential')(dense1)

# define model
model = Model(inputs = in_layers, outputs = mu)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 1, 2)                 32        ['input_8[0][0]']             
                                                                                                  
 input_7 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 reshape_2 (Reshape)         (None, 2)                    0         ['embedding_2[0][0]']         
                                                                                            

In [64]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [65]:
model.fit(x_train, y_train, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 1s - loss: 0.9969 - 924ms/epoch - 7ms/step
Epoch 2/100
140/140 - 0s - loss: 0.4685 - 254ms/epoch - 2ms/step
Epoch 3/100
140/140 - 0s - loss: 0.4556 - 370ms/epoch - 3ms/step
Epoch 4/100
140/140 - 0s - loss: 0.4452 - 349ms/epoch - 2ms/step
Epoch 5/100
140/140 - 0s - loss: 0.4374 - 327ms/epoch - 2ms/step
Epoch 6/100
140/140 - 0s - loss: 0.4321 - 350ms/epoch - 3ms/step
Epoch 7/100
140/140 - 0s - loss: 0.4288 - 351ms/epoch - 3ms/step
Epoch 8/100
140/140 - 0s - loss: 0.4268 - 340ms/epoch - 2ms/step
Epoch 9/100
140/140 - 0s - loss: 0.4258 - 389ms/epoch - 3ms/step
Epoch 10/100
140/140 - 0s - loss: 0.4252 - 257ms/epoch - 2ms/step
Epoch 11/100
140/140 - 0s - loss: 0.4250 - 222ms/epoch - 2ms/step
Epoch 12/100
140/140 - 0s - loss: 0.4248 - 208ms/epoch - 1ms/step
Epoch 13/100
140/140 - 0s - loss: 0.4248 - 208ms/epoch - 1ms/step
Epoch 14/100
140/140 - 0s - loss: 0.4247 - 218ms/epoch - 2ms/step
Epoch 15/100
140/140 - 0s - loss: 0.4247 - 220ms/epoch - 2ms/step
Epoch 16/100
140/14

In [66]:
# make predictions
xnum_test = test[numVar]
xcat_test = test[catVar]
x_test = [xnum_test, xcat_test]
yhat = model.predict(x_test)
print(yhat)

598/598 [==============================] - 1s 1ms/step
[[0.08440963]
 [0.15649007]
 [0.11282077]
 ...
 [0.1514711 ]
 [0.14659931]
 [0.1762785 ]]


In [67]:
metrics.mean_squared_error(ytest, yhat)

0.3391066769849685

**Compared to the test data MSE of 0.3419 in question 2, we can conclude that the test data MSE improve slightly by changing the embedding dimension to 2.**

# **Question 4**

**From 2, change number of neurons to 5, does this improve test MSE?**

In [68]:
in_layer = Input(shape = (xtrain.shape[1],))
dense1 = Dense(5, activation = 'tanh')(in_layer)
mu = Dense(1, activation = 'exponential')(dense1)

In [69]:
model = Model(inputs = in_layer, outputs = mu)
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_10 (Dense)            (None, 5)                 20        
                                                                 
 dense_11 (Dense)            (None, 1)                 6         
                                                                 
Total params: 26 (104.00 Byte)
Trainable params: 26 (104.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [71]:
model.fit(xtrain, ytrain, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 1s - loss: 0.5023 - 760ms/epoch - 5ms/step
Epoch 2/100
140/140 - 0s - loss: 0.4724 - 204ms/epoch - 1ms/step
Epoch 3/100
140/140 - 0s - loss: 0.4630 - 204ms/epoch - 1ms/step
Epoch 4/100
140/140 - 0s - loss: 0.4546 - 213ms/epoch - 2ms/step
Epoch 5/100
140/140 - 0s - loss: 0.4448 - 213ms/epoch - 2ms/step
Epoch 6/100
140/140 - 0s - loss: 0.4389 - 217ms/epoch - 2ms/step
Epoch 7/100
140/140 - 0s - loss: 0.4370 - 218ms/epoch - 2ms/step
Epoch 8/100
140/140 - 0s - loss: 0.4362 - 203ms/epoch - 1ms/step
Epoch 9/100
140/140 - 0s - loss: 0.4354 - 213ms/epoch - 2ms/step
Epoch 10/100
140/140 - 0s - loss: 0.4348 - 324ms/epoch - 2ms/step
Epoch 11/100
140/140 - 0s - loss: 0.4343 - 316ms/epoch - 2ms/step
Epoch 12/100
140/140 - 0s - loss: 0.4338 - 284ms/epoch - 2ms/step
Epoch 13/100
140/140 - 0s - loss: 0.4335 - 286ms/epoch - 2ms/step
Epoch 14/100
140/140 - 0s - loss: 0.4333 - 343ms/epoch - 2ms/step
Epoch 15/100
140/140 - 0s - loss: 0.4330 - 288ms/epoch - 2ms/step
Epoch 16/100
140/14

In [72]:
xtest = test[xnames]
ytest = test[ynames]
yhat = model.predict(xtest)
print(yhat)

598/598 [==============================] - 1s 1ms/step
[[0.12097973]
 [0.12157421]
 [0.12097973]
 ...
 [0.12097866]
 [0.12097976]
 [0.1209797 ]]


In [73]:
metrics.mean_squared_error(ytest, yhat)

0.3436389320368015

**Based on the outputs above, we can found that the test data MSE didn't get improved even by changing the number of neurons from 2 to 5.**

# **Question 5**

**From 2, change number of hidden layers to 2, does this improve test MSE?**

In [81]:
in_layer = Input(shape = (xtrain.shape[1],))
dense1 = Dense(2, activation = 'tanh')(in_layer)
dense2 = Dense(2, activation = 'tanh')(dense1)
mu = Dense(1, activation = 'exponential')(dense2)

In [82]:
model = Model(inputs = in_layer, outputs = mu)
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_18 (Dense)            (None, 2)                 8         
                                                                 
 dense_19 (Dense)            (None, 2)                 6         
                                                                 
 dense_20 (Dense)            (None, 1)                 3         
                                                                 
Total params: 17 (68.00 Byte)
Trainable params: 17 (68.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [83]:
model.compile(optimizer = 'adam', loss = 'Poisson')

In [84]:
model.fit(xtrain, ytrain, epochs=100, shuffle=False, batch_size=128, verbose=2)

Epoch 1/100
140/140 - 1s - loss: 1.0608 - 877ms/epoch - 6ms/step
Epoch 2/100
140/140 - 0s - loss: 0.5857 - 203ms/epoch - 1ms/step
Epoch 3/100
140/140 - 0s - loss: 0.5029 - 198ms/epoch - 1ms/step
Epoch 4/100
140/140 - 0s - loss: 0.4681 - 200ms/epoch - 1ms/step
Epoch 5/100
140/140 - 0s - loss: 0.4514 - 212ms/epoch - 2ms/step
Epoch 6/100
140/140 - 0s - loss: 0.4434 - 225ms/epoch - 2ms/step
Epoch 7/100
140/140 - 0s - loss: 0.4393 - 200ms/epoch - 1ms/step
Epoch 8/100
140/140 - 0s - loss: 0.4372 - 209ms/epoch - 1ms/step
Epoch 9/100
140/140 - 0s - loss: 0.4361 - 200ms/epoch - 1ms/step
Epoch 10/100
140/140 - 0s - loss: 0.4356 - 203ms/epoch - 1ms/step
Epoch 11/100
140/140 - 0s - loss: 0.4352 - 208ms/epoch - 1ms/step
Epoch 12/100
140/140 - 0s - loss: 0.4350 - 200ms/epoch - 1ms/step
Epoch 13/100
140/140 - 0s - loss: 0.4348 - 207ms/epoch - 1ms/step
Epoch 14/100
140/140 - 0s - loss: 0.4346 - 204ms/epoch - 1ms/step
Epoch 15/100
140/140 - 0s - loss: 0.4344 - 196ms/epoch - 1ms/step
Epoch 16/100
140/14

In [85]:
xtest = test[xnames]
ytest = test[ynames]
yhat = model.predict(xtest)
print(yhat)

598/598 [==============================] - 1s 2ms/step
[[0.11792786]
 [0.12025871]
 [0.11792786]
 ...
 [0.11792786]
 [0.11792822]
 [0.11792786]]


In [86]:
metrics.mean_squared_error(ytest, yhat)

0.34452408300512094

**The test data MSE equals to 0.3445, we can conclude that adding one more hidden layer doesn't help us improve the test MSE.**